In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

DATA_PATH = "data"
DB_FAISS_PATH = "vectorstores/db_chrome"

In [2]:
!pip install pypdf

In [3]:
loader = DirectoryLoader(DATA_PATH , glob = '*.pdf' , loader_cls = PyPDFLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
texts = text_splitter.split_documents(documents)

In [4]:
print(loader)

In [5]:
print(documents)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2' , model_kwargs = {'device' : 'cpu'})

In [7]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={} multi_process=False


In [8]:
!pip install FAISS

ERROR: Could not find a version that satisfies the requirement FAISS (from versions: none)
ERROR: No matching distribution found for FAISS


In [9]:
!python --version

Python 3.10.12


In [10]:
persist_directory = "vectorstores\db_chroma"

vectordb = Chroma.from_documents(documents = texts ,
                                 embedding = embeddings ,
                                persist_directory = persist_directory)

In [ ]:
!pip install chromadb

In [11]:
vectordb.persist()
vectordb = None

In [12]:
vectordb = Chroma(persist_directory = persist_directory ,
                 embedding_function = embeddings)

In [13]:
vectordb

In [23]:
import zipfile
import os
import logging

def zip_folder(folder_path, zip_path):
    # Configure logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    logger.info(f"Zipping folder: {folder_path}")
    try:
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, folder_path)
                    zipf.write(file_path, arcname=arcname)
                    logger.info(f"Added {file_path} to the zip file")

        logger.info(f"Zip file created successfully: {zip_path}")
    except Exception as e:
        logger.error(f"Error creating zip file: {e}")
        raise

# Example usage
folder_to_zip = '/content/vectorstores\db_chroma'
zip_file_path = '/content/mybitch'

zip_folder(folder_to_zip, zip_file_path)